# Damage function calibration

In [1]:
%matplotlib widget
import matplotlib.pylab as plt
from ibvpy.bcond import BCSlice
from ibvpy.fets import FETS3D8H
from ibvpy.tmodel.mats3D import MATS3DElastic, MATS3DScalarDamage
import bmcs_matmod.ms1.concrete_material_db as mp_db
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.sim.tstep_bc import TStepBC
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid

import numpy as np

## Current state damage function calibration
 * Setup the single element unit volume problem
 
## Todo
 * Define a strain-hardening damage function for smeared reinforcement model

In [34]:
from bmcs_expsim.utils.mlab_decorators import decorate_figure

In [35]:
size = 1
n_e = 1
xmodel = XDomainFEGrid(coord_min=(0,0,0),
                     coord_max=(size , size, size),
                     shape=(n_e, n_e, n_e),
                     integ_factor=1,
                     fets=FETS3D8H())

In [23]:
tmodel = MATS3DScalarDamage(E=28000, nu=0.3, epsilon_0=3.5 / 28000, epsilon_f=0.001)
tmodel.epsilon_0, tmodel.epsilon_f

(0.000125, 0.001)

In [24]:
w_max = 0.02 * size
#w_max = -1
bc_l_x = BCSlice(slice=xmodel.mesh[0, :, :, 0, :, :],
                  var='u', dims=[0], value=0.0)
bc_r_x = BCSlice(slice=xmodel.mesh[-1, :, :, -1, :, :],
                 var='u', dims=[0], value=w_max)
bc_xy_z0 = BCSlice(slice=xmodel.mesh[:, :, 0, :, :, 0],
                  var='u', dims=[2], value=0)
bc_xz_y0 = BCSlice(slice=xmodel.mesh[:, 0, :, :, 0, :],
                 var='u', dims=[1], value=0)

In [30]:
m = TStepBC(
    domains=[(xmodel, tmodel)],
    bc=[bc_l_x, bc_r_x, bc_xy_z0, bc_xz_y0] # , bc_r_y],
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    # 'stress': Vis3DTensorField(var='sig_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

In [31]:
w_steps = 50
s = m.sim
s.tloop.k_max = 1000
s.tline.step = 1 / w_steps
s.tloop.verbose = True
s.run()

t:	 0.00(0), 
	 0.02(1), 
	 0.04(1), 
	 0.06(1), 
	 0.08(1), 
	 0.10(1), 
	 0.12(1), 
	 0.14(1), 
	 0.16(1), 
	 0.18(1), 
	 0.20(1), 
	 0.22(1), 
	 0.24(1), 
	 0.26(1), 
	 0.28(1), 
	 0.30(1), 
	 0.32(1), 
	 0.34(1), 
	 0.36(0), 
	 0.38(0), 
	 0.40(0), 
	 0.42(0), 
	 0.44(0), 
	 0.46(0), 
	 0.48(0), 
	 0.50(0), 
	 0.52(0), 
	 0.54(0), 
	 0.56(0), 
	 0.58(0), 
	 0.60(0), 
	 0.62(0), 
	 0.64(0), 
	 0.66(0), 
	 0.68(0), 
	 0.70(0), 
	 0.72(0), 
	 0.74(0), 
	 0.76(0), 
	 0.78(0), 
	 0.80(0), 
	 0.82(0), 
	 0.84(0), 
	 0.86(0), 
	 0.88(0), 
	 0.90(0), 
	 0.92(0), 
	 0.94(0), 
	 0.96(0), 
	 0.98(0), 
	 1.00(0), 



In [32]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [33]:
_, ax = plt.subplots(1,1)
F_l_t = np.sum(F_to[:, bc_l_x.dofs], axis=-1)
F_r_t = np.sum(F_to[:, bc_r_x.dofs], axis=-1)
U_r_t = np.average(U_to[:, bc_r_x.dofs], axis=-1)

ax.plot(U_r_t, F_r_t)
ax.plot(U_r_t, -F_l_t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:

#mlab.options.backend = 'envisage'

f_strain = mlab.figure()
scene = mlab.get_engine().scenes[-1]
scene.name = 'strain'
strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
strain_viz.setup()

decorate_figure(f_strain, strain_viz, 200, [70, 20, 0])

# f_stress = mlab.figure()
# scene = mlab.get_engine().scenes[-1]
# scene.name = 'stress'
# stress_viz = Viz3DTensorField(vis3d=m.hist['stress'])
# stress_viz.setup()
#
# decorate_figure(f_stress, stress_viz, 200, [70, 20, 0])

mlab.show()

NameError: name 'mlab' is not defined